; Chat Demo - Fixed Query Demonstration
; =====================================
; This file runs a fixed set of queries to demonstrate the ReAct agent.
; For interactive use, see chat.lisp instead.
;
; Usage (interactive):
;   cd src
;   acl2
;   (ld "chat-demo.lisp")
;
; Usage (certification):
;   cert.pl chat-demo --acl2-cmd 'env SKIP_INTERACTIVE=1 acl2'
;   or just: cert.pl chat-demo  (uses cert_env below)

In [65]:
(in-package "ACL2")

 "ACL2"


In [66]:
(include-book "chat-lib"
              :ttags ((:quicklisp) (:quicklisp.osicat) (:quicklisp.dexador) 
                      (:http-json) (:llm-client) (:mcp-client)))


The event ( INCLUDE-BOOK "chat-lib" ...) is redundant.  See :DOC redundant-
events.

Summary
Form:  ( INCLUDE-BOOK "chat-lib" ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 :REDUNDANT


In [67]:
#-skip-interactive
(make-event
 (mv-let (err models state)
   (llm-list-models-full state)
   (prog2$ 
    (if err
        (cw "~%Error getting models: ~s0~%" err)
      (progn$
       (cw "~%Available models: ~x0~%" (len models))
       (cw "Loaded completions models: ~x0~%" 
           (len (filter-loaded-completions-models models)))))
    (mv nil '(value-triple :models-listed) state))))


Available models: 15
Loaded completions models: 1

Summary
Form:  ( MAKE-EVENT (MV-LET ...))
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 :MODELS-LISTED


In [68]:
#-skip-interactive
(make-event
 (mv-let (err models state)
   (llm-list-models-full state)
   (declare (ignorable err))
   (let ((selected (select-completions-model models *model-prefs*)))
     (prog2$
      (if selected
          (cw "~%Selected model: ~s0 (context: ~x1 tokens)~%"
              (model-info->id selected)
              (model-info->loaded-context-length selected))
        (cw "~%WARNING: No suitable model found! Make sure LM Studio is running.~%"))
      (mv nil 
          `(defconst *model-id* 
             ,(if selected (model-info->id selected) "no-model-found"))
          state)))))


Selected model: qwen/qwen3-coder-30b (context: 80000 tokens)

The event ( DEFCONST *MODEL-ID* ...) is redundant.  See :DOC redundant-
events.

Summary
Form:  ( DEFCONST *MODEL-ID* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)

Summary
Form:  ( MAKE-EVENT (MV-LET ...))
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 :REDUNDANT


In [69]:
(defconst *demo-agent-v0*
  (init-agent-conversation *default-system-prompt* *default-agent-config*))


The event ( DEFCONST *DEMO-AGENT-V0* ...) is redundant.  See :DOC redundant-
events.

Summary
Form:  ( DEFCONST *DEMO-AGENT-V0* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 :REDUNDANT


In [70]:
#-skip-interactive
(defconst *check-init*
  (prog2$ (cw "~%Agent initialized. Messages: ~x0~%" 
              (len (get-messages *demo-agent-v0*)))
          t))


Agent initialized. Messages: 1

The event ( DEFCONST *CHECK-INIT* ...) is redundant.  See :DOC redundant-
events.

Summary
Form:  ( DEFCONST *CHECK-INIT* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 :REDUNDANT


In [71]:
#-skip-interactive
(defconst *demo-agent-v1*
  (add-user-msg "Hello! Can you explain what a formally verified agent is?" 
                *demo-agent-v0*))


The event ( DEFCONST *DEMO-AGENT-V1* ...) is redundant.  See :DOC redundant-
events.

Summary
Form:  ( DEFCONST *DEMO-AGENT-V1* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 :REDUNDANT


In [72]:
#-skip-interactive
(defconst *show-v1*
  (show-conversation *demo-agent-v1*))


The event ( DEFCONST *SHOW-V1* ...) is redundant.  See :DOC redundant-
events.

Summary
Form:  ( DEFCONST *SHOW-V1* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 :REDUNDANT


In [73]:
#-skip-interactive
(make-event
 (mv-let (err response state)
   (llm-chat-completion *model-id* (get-messages *demo-agent-v1*) state)
   (if err
       (prog2$ (cw "~%LLM Error: ~s0~%" err)
               (mv nil `(defconst *demo-agent-v2* *demo-agent-v1*) state))
     (prog2$ (cw "~%Assistant: ~s0~%" response)
             (mv nil `(defconst *demo-agent-v2* 
                        (add-assistant-msg ,response *demo-agent-v1*)) 
                 state)))))


Assistant: A formally verified agent is an AI system whose behavior
and decisions can be mathematically proven to satisfy specific properties
or specifications. Let me break this down:

1. **Formal Verification**: This is a rigorous mathematical process
where we prove that a system's behavior matches its intended specification
using logical reasoning and mathematical proofs.

2. **Agent**: In AI context, this refers to a system that perceives
its environment and takes actions to achieve goals.

3. **Combined**: A formally verified agent means we can mathematically
guarantee that it will behave correctly according to its design specifications\
 under all circumstances.

In the context of ACL2 (a formal verification system), this means:
- The agent's logic is encoded in ACL2's mathematical framework
- We can prove properties about its behavior using ACL2's theorem proving
capabilities
- The system's operations are verified to meet specified requirements

This is important because it pro

In [74]:
#-skip-interactive
(defconst *show-v2*
  (show-conversation *demo-agent-v2*))


========== Conversation ==========
[SYSTEM]
You are a helpful AI assistant running inside a formally verified ReAct
agent framework built in ACL2.

You have access to ACL2 code execution. To run ACL2 code, put it in
a fenced code block:

```acl2
(+ 1 2 3)
```

or:

```lisp
(defun factorial (n) (if (zp n) 1 (* n (factorial (1- n)))))
```

I will execute the code and show you the result. You can then continue
reasoning or give a final answer.

Be concise. Show your reasoning.

[USER]
Hello! Can you explain what a formally verified agent is?

[ASSISTANT]
A formally verified agent is an AI system whose behavior and decision-
making processes have been mathematically proven to satisfy specific
properties or requirements. This involves using formal methods - rigorous
mathematical techniques - to demonstrate that the agent behaves correctly
under all possible conditions, rather than just testing it empirically.

The key aspects include:

1. **Formal Specification**: Precise mathematical desc

In [75]:
#-skip-interactive
(defconst *demo-agent-v3*
  (add-user-msg "What properties have been proven about this agent?" 
                *demo-agent-v2*))


The event ( DEFCONST *DEMO-AGENT-V3* ...) is redundant.  See :DOC redundant-
events.

Summary
Form:  ( DEFCONST *DEMO-AGENT-V3* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 :REDUNDANT


In [76]:
#-skip-interactive
(make-event
 (mv-let (err response state)
   (llm-chat-completion *model-id* (get-messages *demo-agent-v3*) state)
   (if err
       (prog2$ (cw "~%LLM Error: ~s0~%" err)
               (mv nil `(defconst *demo-agent-v4* *demo-agent-v3*) state))
     (prog2$ (cw "~%Assistant: ~s0~%" response)
             (mv nil `(defconst *demo-agent-v4* 
                        (add-assistant-msg ,response *demo-agent-v3*)) 
                 state)))))


Assistant: Looking at this specific agent, I can see it's running in
an ACL2 environment, which is a formal verification system. However,
I don't have access to the specific proof certificates or formal properties
that have been established for this particular agent instance.

In general, for an ACL2-based formally verified agent like this one,
typical properties that might be proven include:

1. **Correctness of computations** - That arithmetic and logical operations
produce mathematically correct results
2. **Safety properties** - That the agent won't crash or enter invalid
states
3. **Specification adherence** - That the agent's behavior matches
its formal specification
4. **Consistency** - That the agent maintains internal logical consistency

Since this is a ReAct agent framework, it would also likely have properties
proven about:
- Proper handling of the reasoning and action cycle
- Correct interpretation of user inputs
- Valid execution of actions within its capabilities

Howev

In [77]:
#-skip-interactive
(defconst *show-v4*
  (show-conversation *demo-agent-v4*))


========== Conversation ==========
[SYSTEM]
You are a helpful AI assistant running inside a formally verified ReAct
agent framework built in ACL2.

You have access to ACL2 code execution. To run ACL2 code, put it in
a fenced code block:

```acl2
(+ 1 2 3)
```

or:

```lisp
(defun factorial (n) (if (zp n) 1 (* n (factorial (1- n)))))
```

I will execute the code and show you the result. You can then continue
reasoning or give a final answer.

Be concise. Show your reasoning.

[USER]
Hello! Can you explain what a formally verified agent is?

[ASSISTANT]
A formally verified agent is an AI system whose behavior and decision-
making processes have been mathematically proven to satisfy specific
properties or requirements. This involves using formal methods - rigorous
mathematical techniques - to demonstrate that the agent behaves correctly
under all possible conditions, rather than just testing it empirically.

The key aspects include:

1. **Formal Specification**: Precise mathematical desc

In [78]:
#-skip-interactive
(defconst *demo-agent-v5*
  (add-user-msg "That's impressive! How does the context management work?" 
                *demo-agent-v4*))


The event ( DEFCONST *DEMO-AGENT-V5* ...) is redundant.  See :DOC redundant-
events.

Summary
Form:  ( DEFCONST *DEMO-AGENT-V5* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 :REDUNDANT


In [79]:
#-skip-interactive
(make-event
 (mv-let (err response state)
   (llm-chat-completion *model-id* (get-messages *demo-agent-v5*) state)
   (if err
       (prog2$ (cw "~%LLM Error: ~s0~%" err)
               (mv nil `(defconst *demo-agent-v6* *demo-agent-v5*) state))
     (prog2$ (cw "~%Assistant: ~s0~%" response)
             (mv nil `(defconst *demo-agent-v6* 
                        (add-assistant-msg ,response *demo-agent-v5*)) 
                 state)))))


Assistant: In a formally verified ReAct agent, context management is
handled through formal specifications that ensure the agent maintains
consistent and correct state across interactions. Let me demonstrate
how this might work in ACL2:

```acl2
(defun context-state (messages history)
  "Formal representation of agent context state"
  (list :messages messages :history history))

(defun update-context (context new-message)
  "Update context with new message while maintaining formal properties"
  (let ((new-messages (cons new-message (getf context :messages)))
        (new-history (cons (getf context :messages) (getf context :history))))\

    (context-state new-messages new-history)))

(defun validate-context (context)
  "Formal validation of context integrity"
  (and (listp (getf context :messages))
       (listp (getf context :history))
       (equal (length (getf context :messages))
              (length (getf context :history)))))
```

The key formal properties ensured are:
1. **St

In [80]:
#-skip-interactive
(defconst *show-v6*
  (show-conversation *demo-agent-v6*))


The event ( DEFCONST *SHOW-V6* ...) is redundant.  See :DOC redundant-
events.

Summary
Form:  ( DEFCONST *SHOW-V6* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 :REDUNDANT


In [81]:
#-skip-interactive
(defconst *context-check*
  (show-context-usage *demo-agent-v6*))


Context Usage:
  Total characters: 3604
  Estimated tokens: 901
  Max tokens: 8000
  Fits in context: T

The event ( DEFCONST *CONTEXT-CHECK* ...) is redundant.  See :DOC redundant-
events.

Summary
Form:  ( DEFCONST *CONTEXT-CHECK* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 :REDUNDANT


In [82]:
#-skip-interactive
(defconst *demo-agent-with-tool*
  (add-tool-result 
    "File contents of verified-agent.lisp (first 500 chars):
; Verified ReAct Agent - ACL2 Implementation
; This file implements a formally verified ReAct agent...
(in-package \"ACL2\")
(include-book \"std/util/define\" :dir :system)
..." 
    *demo-agent-v6*))


The event ( DEFCONST *DEMO-AGENT-WITH-TOOL* ...) is redundant.  See
:DOC redundant-events.

Summary
Form:  ( DEFCONST *DEMO-AGENT-WITH-TOOL* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 :REDUNDANT


In [83]:
#-skip-interactive
(defconst *show-tool*
  (show-conversation *demo-agent-with-tool*))


========== Conversation ==========
[SYSTEM]
You are a helpful AI assistant running inside a formally verified ReAct
agent framework built in ACL2.

You have access to ACL2 code execution. To run ACL2 code, put it in
a fenced code block:

```acl2
(+ 1 2 3)
```

or:

```lisp
(defun factorial (n) (if (zp n) 1 (* n (factorial (1- n)))))
```

I will execute the code and show you the result. You can then continue
reasoning or give a final answer.

Be concise. Show your reasoning.

[USER]
Hello! Can you explain what a formally verified agent is?

[ASSISTANT]
A formally verified agent is an AI system whose behavior and decision-
making processes have been mathematically proven to satisfy specific
properties or requirements. This involves using formal methods - rigorous
mathematical techniques - to demonstrate that the agent behaves correctly
under all possible conditions, rather than just testing it empirically.

The key aspects include:

1. **Formal Specification**: Precise mathematical desc

In [84]:
(defconst *read-file-tool*
  (make-tool-spec
    :name 'read-file
    :required-access 1
    :requires-execute nil
    :token-cost 100
    :time-cost 5))


The event ( DEFCONST *READ-FILE-TOOL* ...) is redundant.  See :DOC
redundant-events.

Summary
Form:  ( DEFCONST *READ-FILE-TOOL* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 :REDUNDANT


In [85]:
#-skip-interactive
(defconst *can-read-file*
  (prog2$ (cw "~%Can invoke read-file tool: ~x0~%" 
              (can-invoke-tool-p *read-file-tool* *demo-agent-v6*))
          (can-invoke-tool-p *read-file-tool* *demo-agent-v6*)))


Can invoke read-file tool: T

The event ( DEFCONST *CAN-READ-FILE* ...) is redundant.  See :DOC redundant-
events.

Summary
Form:  ( DEFCONST *CAN-READ-FILE* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 :REDUNDANT


In [86]:
#-skip-interactive
(defconst *demo-agent-after-react*
  (if (and (not (must-respond-p *demo-agent-v6*))
           (can-invoke-tool-p *read-file-tool* *demo-agent-v6*))
      (react-step-with-conversation
        (agent-action-tool-call 'read-file "verified-agent.lisp")
        *read-file-tool*
        "I'll read the verified-agent.lisp file to show you the implementation."
        "File read successfully. Contents: (in-package ACL2)..."
        *demo-agent-v6*)
    *demo-agent-v6*))


The event ( DEFCONST *DEMO-AGENT-AFTER-REACT* ...) is redundant.  See
:DOC redundant-events.

Summary
Form:  ( DEFCONST *DEMO-AGENT-AFTER-REACT* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 :REDUNDANT


In [87]:
#-skip-interactive
(defconst *show-react*
  (prog2$ (cw "~%After ReAct step:~%")
    (prog2$ (cw "  Step counter: ~x0~%" 
                (agent-state->step-counter *demo-agent-after-react*))
      (prog2$ (cw "  Token budget: ~x0~%" 
                  (agent-state->token-budget *demo-agent-after-react*))
        (cw "  Message count: ~x0~%" 
            (len (get-messages *demo-agent-after-react*)))))))


After ReAct step:
  Step counter: 1
  Token budget: 49900
  Message count: 9

The event ( DEFCONST *SHOW-REACT* ...) is redundant.  See :DOC redundant-
events.

Summary
Form:  ( DEFCONST *SHOW-REACT* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 :REDUNDANT


In [88]:
#-skip-interactive
(defconst *demo-complete*
  (prog2$ (cw "~%~%========================================~%")
    (prog2$ (cw "Chat Demo Complete!~%")
      (prog2$ (cw "========================================~%")
        (prog2$ (cw "~%Key points demonstrated:~%")
          (prog2$ (cw "  1. Agent state initialization~%")
            (prog2$ (cw "  2. Conversation management (add-user-msg, add-assistant-msg)~%")
              (prog2$ (cw "  3. Context length tracking~%")
                (prog2$ (cw "  4. Tool result handling~%")
                  (prog2$ (cw "  5. ReAct step with conversation~%")
                    (prog2$ (cw "~%For interactive chat, use chat.lisp instead:~%")
                      (prog2$ (cw "  (ld \"chat.lisp\")~%")
                        (cw "  (chat \"your message\")~%")))))))))))))



Chat Demo Complete!

Key points demonstrated:
  1. Agent state initialization
  2. Conversation management (add-user-msg, add-assistant-msg)
  3. Context length tracking
  4. Tool result handling
  5. ReAct step with conversation

For interactive chat, use chat.lisp instead:
  (ld "chat.lisp")
  (chat "your message")

The event ( DEFCONST *DEMO-COMPLETE* ...) is redundant.  See :DOC redundant-
events.

Summary
Form:  ( DEFCONST *DEMO-COMPLETE* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 :REDUNDANT
